In [79]:
import numpy as np
import pandas as pd
from IPython.display import display
import csv
from PIL import Image
from scipy.ndimage import rotate
import tensorflow as tf
from tensorflow import keras as ks
%matplotlib inline

In [80]:
letters_training_images_file_path = "/home/mohamed/Downloads/archive/csvTrainImages 13440x1024.csv"
letters_training_labels_file_path = "/home/mohamed/Downloads/archive/csvTrainLabel 13440x1.csv"
letters_testing_images_file_path = "/home/mohamed/Downloads/archive/csvTestImages 3360x1024.csv"
letters_testing_labels_file_path = "/home/mohamed/Downloads/archive/csvTestLabel 3360x1.csv"


training_letters_images = pd.read_csv(letters_training_images_file_path, header=None)
training_letters_labels = pd.read_csv(letters_training_labels_file_path, header=None)
testing_letters_images = pd.read_csv(letters_testing_images_file_path, header=None)
testing_letters_labels = pd.read_csv(letters_testing_labels_file_path, header=None)

# print statistics about the dataset
print("There are %d training arabic letter images of 32x32 pixels." %training_letters_images.shape[0])
print("There are %d testing arabic letter images of 32x32 pixels." %testing_letters_images.shape[0])
training_letters_images.head()

There are 13440 training arabic letter images of 32x32 pixels.
There are 3360 testing arabic letter images of 32x32 pixels.


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
def convert_values_to_image(image_values, display=False):
    image_array = np.asarray(image_values)
    image_array = image_array.reshape(32,32).astype('uint8')
    # The original dataset is reflected so we will flip it then rotate for a better view only.
    image_array = np.flip(image_array, 0)
    image_array = rotate(image_array, -90)
    new_image = Image.fromarray(image_array)
    if display == True:
        new_image.show()
    return new_image

In [82]:
training_letters_images_scaled = training_letters_images.values.astype('float32')/255
training_letters_labels = training_letters_labels.values.astype('int32')
testing_letters_images_scaled = testing_letters_images.values.astype('float32')/255
testing_letters_labels = testing_letters_labels.values.astype('int32')

In [83]:
print("Training images of letters after scaling")
print(training_letters_images_scaled.shape)
training_letters_images_scaled[0:5]

Training images of letters after scaling
(13440, 1024)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [84]:
number_of_classes = 28

training_letters_labels_encoded = ks.utils.to_categorical(training_letters_labels-1, num_classes=number_of_classes)
testing_letters_labels_encoded = ks.utils.to_categorical(testing_letters_labels-1, num_classes=number_of_classes)

In [85]:
print(training_letters_labels_encoded)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [86]:
training_letters_images_scaled = training_letters_images_scaled.reshape([-1, 32, 32, 1])
testing_letters_images_scaled = testing_letters_images_scaled.reshape([-1, 32, 32, 1])

print(training_letters_images_scaled.shape, training_letters_labels_encoded.shape, testing_letters_images_scaled.shape, testing_letters_labels_encoded.shape)

(13440, 32, 32, 1) (13440, 28) (3360, 32, 32, 1) (3360, 28)


In [90]:
def create_model(optimizer='adam', activation='relu'):
    # create model
    model = ks.models.Sequential()
    model.add(ks.layers.Conv2D(filters=16, kernel_size=3, padding='same', input_shape=(32, 32, 1), activation=activation))
    model.add(ks.layers.BatchNormalization())
    model.add(ks.layers.MaxPooling2D(pool_size=2))
    model.add(ks.layers.Dropout(0.2))

    model.add(ks.layers.Conv2D(filters=32, kernel_size=3, padding='same',activation=activation))
    model.add(ks.layers.BatchNormalization())
    model.add(ks.layers.MaxPooling2D(pool_size=2))
    model.add(ks.layers.Dropout(0.2))

    model.add(ks.layers.Conv2D(filters=64, kernel_size=3, padding='same',activation=activation))
    model.add(ks.layers.BatchNormalization())
    model.add(ks.layers.MaxPooling2D(pool_size=2))
    model.add(ks.layers.Dropout(0.2))

    model.add(ks.layers.Conv2D(filters=128, kernel_size=3, padding='same',activation=activation))
    model.add(ks.layers.BatchNormalization())
    model.add(ks.layers.MaxPooling2D(pool_size=2))
    model.add(ks.layers.Dropout(0.2))
    model.add(ks.layers.GlobalAveragePooling2D())

    model.add(ks.layers.Dense(100, activation='relu'))
    model.add(ks.layers.Dense(75, activation='relu'))
    model.add(ks.layers.Dense(50, activation='relu'))
    model.add(ks.layers.Dense(28, activation='softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)
    
    return model

In [91]:
model = create_model()
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 32, 32, 16)        160       
_________________________________________________________________
batch_normalization_26 (Batc (None, 32, 32, 16)        64        
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
batch_normalization_27 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 8, 8, 32)         

In [93]:
model = create_model(optimizer='Adam', activation='relu')

In [94]:
history = model.fit(training_letters_images_scaled, training_letters_labels_encoded, 
                    validation_data=(testing_letters_images_scaled, testing_letters_labels_encoded),
                    epochs=15, batch_size=64)

Epoch 1/15
210/210 [==============================] - 20s 92ms/step - loss: 2.7350 - accuracy: 0.1850 - val_loss: 5.6664 - val_accuracy: 0.0357
Epoch 2/15
210/210 [==============================] - 20s 98ms/step - loss: 0.9009 - accuracy: 0.6815 - val_loss: 4.8182 - val_accuracy: 0.0714
Epoch 3/15
210/210 [==============================] - 22s 104ms/step - loss: 0.5865 - accuracy: 0.7991 - val_loss: 0.4699 - val_accuracy: 0.8461
Epoch 4/15
210/210 [==============================] - 23s 107ms/step - loss: 0.4482 - accuracy: 0.8469 - val_loss: 0.5130 - val_accuracy: 0.8152
Epoch 5/15
210/210 [==============================] - 21s 101ms/step - loss: 0.3800 - accuracy: 0.8676 - val_loss: 0.4869 - val_accuracy: 0.8464
Epoch 6/15
210/210 [==============================] - 22s 106ms/step - loss: 0.3280 - accuracy: 0.8895 - val_loss: 0.4043 - val_accuracy: 0.8619
Epoch 7/15
210/210 [==============================] - 27s 129ms/step - loss: 0.2876 - accuracy: 0.9017 - val_loss: 0.2220 - val_accu

In [96]:
np.random.seed(0)

optimizer = ['SGD','RMSprop', 'Adam', 'Adagrad', 'Nadam']
activation = 'relu'

param_grid = dict(optimizer=optimizer)

# count number of different parameters values combinations
parameters_number = 1
for x in param_grid:
    parameters_number = parameters_number * len(param_grid[x]) 
print("Number of different parameter combinations = {}".format(parameters_number))

Number of different parameter combinations = 5
